#Librerias

In [1]:
!pip install ipythonblocks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive/')
sys.path.append('/content/drive/My Drive/UPC/10_topicos/TA4')
from agents import *
import random
import numpy as np

Mounted at /content/drive/


#Código

In [8]:
class Food(Thing):
  pass
class Water(Thing):
  pass

In [4]:
def mainprogram(detects):
  ubi_pasadas=[]
  for p in detects:
    if p.location not in ubi_pasadas:
      ubi_pasadas.append(p.location)
      return random.choice(["move up", "move left", "move right","move down"])

    else:
        if p.action == "move down":
          ubi_pasadas.append(p.location)
          return random.choice(["move up", "move left", "move right"])
        elif p.action == "move up":
          ubi_pasadas.append(p.location)
          return random.choice(["move down", "move left", "move right"])
        elif p.action == "move left":
          ubi_pasadas.append(p.location)
          return random.choice(["move up", "move down", "move right"])
        elif p.action == "move right":
          ubi_pasadas.append(p.location)
          return random.choice(["move up", "move down", "move left"])

    if isinstance(p, Food):
      ubi_pasadas.append(p.location)
      return "eat"
    elif isinstance(p, Water):
      ubi_pasadas.append(p.location)
      return "drink"

In [5]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things

    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "move up":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.moveup()
        elif action == "move right":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.moveright()
        elif action == "move left":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.moveleft()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent,
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [6]:
class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing

    def movedown(self):
        self.location[1] += 1

    def moveup(self):
        self.location[1] -= 1

    def moveright(self):
        self.location[0] += 1

    def moveleft(self):
        self.location[0] -= 1

    def move_towards(self, target_location):
        # Calculate the direction to move
        direction = np.sign(np.array(target_location) - np.array(self.location))
        # Update the location
        self.location = (np.array(self.location) + direction).tolist()

    def move(self, park):
        # Find the closest Food or Water
        closest_target = None
        min_distance = float('inf')
        for thing in park.things:
            if isinstance(thing, (Food, Water)):
                distance = np.linalg.norm(np.array(thing.location) - np.array(self.location))
                if distance < min_distance:
                    min_distance = distance
                    closest_target = thing.location

        if closest_target is not None:
            self.move_towards(closest_target)
        else:
            # Random move if no target found
            self.location = [self.location[0] + random.choice([-1, 1]), self.location[1] + random.choice([-1, 1])]

        # Ensure the dog stays within bounds
        self.location[0] = max(0, min(self.location[0], park.width - 1))
        self.location[1] = max(0, min(self.location[1], park.height - 1))

    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False

    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

In [7]:
park = Park2D(30,30, color={'BlindDog':(200,0,0),'Water':(0,200,200),'Food':(230,100,40)})

for i in range(3):
  dog_0 = BlindDog(mainprogram)
  park.add_thing(dog_0, [random.randint(0, 29),random.randint(0, 29)])

for i in range(7):
  water_0 = Water()
  food_0 = Food()
  park.add_thing(water_0, [random.randint(0, 29),random.randint(0, 29)])
  park.add_thing(food_0,  [random.randint(0, 29),random.randint(0, 29)])

park.run(40)

Output hidden; open in https://colab.research.google.com to view.